In [12]:
import networkx as nx
import qml
import igraph as ig
import graph_tool as gt
import graph_tool.topology as gtt
import MDAnalysis as mda
import numpy as np

In [108]:
class CompareNX(object):
    def __init__(self, mol2file):
        bonds = mda.topology.MOL2Parser.MOL2Parser(mol2file).parse().bonds.values
        self._g1 = nx.Graph()
        self._g2 = nx.Graph()
        for b1, b2 in bonds:
            self._g1.add_edge(b1, b2)
            self._g2.add_edge(b1, b2)
        self._length = len(self._g1.nodes)
        self._match = nx.algorithms.isomorphism.categorical_node_match('element', 1)
        
    def compare(self, n1, n2):
        for pos in range(self._length):
            self._g1.nodes[pos]['element'] = n1[pos]
            self._g2.nodes[pos]['element'] = n2[pos]
        return nx.algorithms.isomorphism.is_isomorphic(self._g1, self._g2, node_match=self._match)

class CompareQML(object):
    def __init__(self, fn):
        self._c = qml.Compound(fn)
        
    def compare(self, n1, n2):
        r1 = qml.fchl.generate_representation(self._c.coordinates, n1, self._c.natoms)
        r2 = qml.fchl.generate_representation(self._c.coordinates, n2, self._c.natoms)
        return qml.fchl.get_global_kernels(np.array([r1]), np.array([r2]), np.array([2])).flatten()[0] > 0.9999

class CompareIG(object):
    def __init__(self, mol2file):
        bonds = mda.topology.MOL2Parser.MOL2Parser(mol2file).parse().bonds.values
        self._g1 = ig.Graph(bonds)
        #self._g2 = ig.Graph(bonds)
        
    def compare(self, n1, n2):
        return self._g1.isomorphic_vf2(self._g1, color1=n1, color2=n2)

class CompareGT(object):
    def __init__(self, mol2file):
        bonds = mda.topology.MOL2Parser.MOL2Parser(mol2file).parse().bonds.values
        self._g1 = gt.Graph()
        self._g2 = gt.Graph()
        natoms = max([max(bond) for bond in bonds])+1
        self._g1.add_vertex(natoms)
        self._g2.add_vertex(natoms)
        for a, b in bonds:
            self._g1.add_edge(self._g1.vertex(a), self._g1.vertex(b))
            self._g2.add_edge(self._g1.vertex(a), self._g1.vertex(b))
            
    def compare(self, n1, n2):
        prop1 = self._g1.new_vertex_property("int")
        for z, v in zip(self._g1.vertices(), n1): 
            prop1[v] = z
        prop2 = self._g2.new_vertex_property("int")
        for z, v in zip(self._g2.vertices(), n2): 
            prop2[v] = z
        return gtt.isomorphism(self._g1, self._g2, prop1, prop2)

In [109]:
cmps = {'QML': CompareQML("../../test/benzene.xyz"), 'NX': CompareNX("benzene.mol2"), 'IG': CompareIG("benzene.mol2"), 'GT': CompareGT("benzene.mol2")}

for k, cmp in cmps.items():
    print (k)
    %timeit (cmp.compare((6,5,7,7,5,6,1,1,1,1,1,1), (6,7,5,5,7,6,1,1,1,1,1,1)), cmp.compare((6,6,6,6,6,6,1,1,1,1,1,1), (6,6,6,6,6,6,1,1,1,1,1,1)))

QML
103 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
NX
1.63 ms ± 34.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
IG
38.6 µs ± 637 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
GT
2.73 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
# check benzene unique
base = [6, 6, 7, 5, 7, 5,]
rest = [1, 1, 1, 1, 1, 1]
import itertools as it
candidates = list(it.permutations(tuple(base)))
found = [candidates[0]]
cmp = CompareIG("benzene.mol2")
for candidate in candidates[1:]:
    cfull = list(candidate) + rest
    for f in found:
        cf = list(f) + rest
        if cmp.compare(cfull, cf):
            break
    else:
        found.append(candidate)
            
    


In [110]:
cmp = CompareIG("benzene.mol2")
cmp.compare((6,5,7,7,5,6,1,1,1,1,1,1), (6,7,5,5,7,6,1,1,1,1,1,1)), cmp.compare((6,6,6,6,6,6,1,1,1,1,1,1), (6,6,6,6,6,6,1,1,1,1,1,1))

(False, True)

In [114]:
a = np.zeros(3)

In [117]:
np.append(a, (1,2))

array([0., 0., 0., 1., 2.])

In [116]:
a

array([0., 0., 0.])

In [122]:
np.arange(-3, 0)

array([-3, -2, -1])